# Registration
Registers images `data[1:]` to image `data[0]`.
Save the results to a folder given by the *folder* variable.

In [ ]:
import os

# Folder name to save registration transforms
folder = "AAAAAAAAAA"

path = "./oai_zib_mri_train/train"
labelPath = "oai_zib_labelmaps/labelmaps/train"

data = os.listdir(path)
data = data[:60]

In [ ]:
import SimpleITK as sitk
import numpy as np
from utils import resampleSegmentation, registration

transforms = []
minx = miny = minz = 10000
maxx = maxy = maxz = 0

for i, f in enumerate(data):
    print(f'Reading {f}. {i+1}/{len(data)}', end=" ")
    
    pathName = os.path.join(path, f)
                  
    # Histogram matching
    if i != 0:
        image = sitk.HistogramMatching(sitk.CurvatureFlow(sitk.ReadImage(pathName), 
                                                          timeStep = 0.04, 
                                                          numberOfIterations = 10),
                                       fixedImage)

        labels = sitk.ReadImage(os.path.join(labelPath, f))
    
        t = registration(fixedImage, image, labels)
        
        transforms.append(t)
        
        labels = sitk.GetArrayFromImage(resampleSegmentation(labels, fixedImage, t))
        
        idx = np.nonzero(labels != 0)
        minx = min([minx, min(idx[0])])
        maxx = max([maxx, max(idx[0])])
        miny = min([miny, min(idx[1])])
        maxy = max([maxy, max(idx[1])])
        minz = min([minz, min(idx[2])])
        maxz = max([maxz, max(idx[2])])
    else:
        fixedImage = sitk.CurvatureFlow(sitk.ReadImage(pathName), timeStep = 0.04, numberOfIterations = 10)
    
    print()

In [ ]:
"""Write transforms."""

from pathlib import Path

Path(f"./{folder}").mkdir(parents=True, exist_ok=True)

file = open(f"./{folder}/names.txt", "w") 

for i, name in enumerate(data):
    print(f"Saving {name}. {i+1}/{len(data)}")
    file.write(name+"\n")
    if i >= 1:
        sitk.WriteTransform(transforms[i-1], f"./{folder}/{name}.tfm")
    
file.close()

file = open(f"./{folder}/boundaries.txt", "w") 
file.write(str(minx)+"\n")
file.write(str(maxx)+"\n")
file.write(str(miny)+"\n")
file.write(str(maxy)+"\n")
file.write(str(minz)+"\n")
file.write(str(maxz)+"\n")
file.close()